In [6]:
import altair as alt
import pandas as pd
import numpy as np

# Load and prepare the data
df = pd.read_csv('us_foreign_aid_country.csv')

# Filter for European and Asian countries
europe_asia_df = df[df['Region Name'].isin(['Europe and Eurasia', 'South and Central Asia', 'East Asia and Oceania'])]

# Clean and convert Fiscal Year to integer
def clean_year(year_str):
    try:
        # Extract first 4 digits (removes any suffixes like 'tq')
        return int(str(year_str)[:4])
    except:
        return np.nan

europe_asia_df['Fiscal Year'] = europe_asia_df['Fiscal Year'].apply(clean_year)
europe_asia_df = europe_asia_df.dropna(subset=['Fiscal Year'])  # Remove rows with invalid years

# Aggregate by region and year
aid_by_region = europe_asia_df.groupby(['Region Name', 'Fiscal Year'])['current_amount'].sum().reset_index()

# Create radio button input for region selection
inputs = alt.binding_radio(options=aid_by_region['Region Name'].unique(), name='Region: ')
selection = alt.selection_point(fields=['Region Name'], bind=inputs, empty=False)

# Get min and max years for axis ticks
min_year = int(aid_by_region['Fiscal Year'].min())
max_year = int(aid_by_region['Fiscal Year'].max())
year_ticks = list(range(min_year, max_year + 1, 10))

# Create scatter plot of aid over time
scatter_plot = alt.Chart(aid_by_region).mark_circle(size=60).encode(
    x=alt.X('Fiscal Year:O', title='Year', axis=alt.Axis(values=year_ticks)),
    y=alt.Y('current_amount:Q', title='Aid Amount (USD)', scale=alt.Scale(type='log')),
    color=alt.condition(selection, 
                       alt.Color('Region Name:N', legend=None),
                       alt.value('lightgray')),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.1)),
    tooltip=['Region Name', 'Fiscal Year', 'current_amount']
).add_params(
    selection
).properties(
    width=800,
    height=400,
    title='US Foreign Aid Over Time by Region (Select Region Below)'
)

# Configure the chart
scatter_plot.configure_axis(
    labelAngle=45
).configure_view(
    strokeWidth=0
).configure_axisX(
    grid=False
).configure_axisY(
    grid=False
)

C:\Users\basil\AppData\Local\Temp\ipykernel_16392\2955182506.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  europe_asia_df['Fiscal Year'] = europe_asia_df['Fiscal Year'].apply(clean_year)


alt.Chart(...)

Webpage Explanation Paragraph:
This interactive visualization reveals trends in US foreign aid allocation to three key regions (East Asia & Oceania, Europe & Eurasia, and South & Central Asia) over time. The logarithmic y-axis allows comparison of aid amounts across different orders of magnitude, while the interactive selection highlights one region at a time for clearer analysis. Key takeaways include the significant increase in aid to Europe/Eurasia post-1990, the steady support for South/Central Asia since 2001, and the relative consistency of aid to East Asia/Oceania. However, in recent years, there is a significant decrease in aid for the South/Central Asia Region and East Asia/Oceania. Users can click the radio buttons to focus on specific regional trends.

Design Rationale (for Word Document):
The visualization employs a minimalist design with three key principles: clarity, comparability, and engagement. The scatter plot format effectively shows distribution and trends over time, while the logarithmic scale accommodates the wide range of aid amounts. The interactive element (region selection) reduces visual clutter by focusing attention on one data series at a time. Color contrast ensures accessibility, and the 10-year tick marks provide temporal context without overcrowding. The design intentionally removes gridlines and borders to direct attention to the data points themselves. The tooltip functionality offers detailed information on demand, maintaining a clean initial view while allowing deeper exploration.

In [12]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html
from dash.dependencies import Input, Output

# Load the data
df = pd.read_csv('us_foreign_aid_funding.csv')

# Data cleaning and preparation
df = df[['Country Name', 'Funding Account Name', 'Transaction Type Name', 'Fiscal Year', 'current_amount']]
df = df[df['current_amount'] > 0]  # Remove negative values

# Categorize countries into the three specified regions
def assign_region(country):
    europe_eurasia = ['Albania', 'Armenia', 'Azerbaijan', 'Belarus', 'Bosnia and Herzegovina',
                     'Bulgaria', 'Croatia', 'Czechia', 'Estonia', 'Georgia', 'Hungary',
                     'Kazakhstan', 'Kosovo', 'Moldova', 'Montenegro', 'North Macedonia',
                     'Russia', 'Serbia', 'Ukraine']
    
    east_asia_oceania = ['China', 'Indonesia', 'Cambodia', 'Timor-Leste']
    
    south_central_asia = ['Afghanistan', 'Bangladesh', 'India', 'Pakistan']
    
    if country in europe_eurasia:
        return 'Europe & Eurasia'
    elif country in east_asia_oceania:
        return 'East Asia & Oceania'
    elif country in south_central_asia:
        return 'South & Central Asia'
    else:
        return None  # Will be filtered out

df['Region'] = df['Country Name'].apply(assign_region)
df = df[df['Region'].notna()]  # Keep only the three specified regions

# Categorize funding by sector
def assign_sector(account_name):
    account_name = str(account_name).lower()
    if 'capital' in account_name or 'investment' in account_name:
        return 'Infrastructure'
    elif 'stabilization' in account_name or 'security' in account_name:
        return 'Security'
    elif 'health' in account_name or 'medical' in account_name:
        return 'Health'
    elif 'education' in account_name or 'school' in account_name:
        return 'Education'
    elif 'aeeca' in account_name or 'democracy' in account_name or 'governance' in account_name:
        return 'Democracy & Governance'
    else:
        return 'Other'

df['Sector'] = df['Funding Account Name'].apply(assign_sector)

# Aggregate data
agg_df = df.groupby(['Region', 'Sector', 'Fiscal Year'])['current_amount'].sum().reset_index()

# Create Dash app
app = Dash(__name__)

app.layout = html.Div([
    html.H1("U.S. Foreign Aid to Key Regions (2004-2025)"),
    html.Div([
        dcc.Dropdown(
            id='year-range',
            options=[{'label': str(year), 'value': year} 
                    for year in sorted(df['Fiscal Year'].unique())],
            value=2023,
            multi=False,
            style={'width': '50%'}
        ),
        dcc.RadioItems(
            id='view-type',
            options=[
                {'label': 'Absolute Values (USD)', 'value': 'absolute'},
                {'label': 'Percentage Composition', 'value': 'percentage'}
            ],
            value='absolute',
            labelStyle={'display': 'inline-block', 'margin-right': '10px'}
        )
    ]),
    dcc.Graph(id='aid-chart'),
    html.Div([
        html.P("Explore U.S. foreign aid distribution across three key regions:"),
        html.Ul([
            html.Li("Europe & Eurasia"),
            html.Li("East Asia & Oceania"),
            html.Li("South & Central Asia")
        ]),
        html.P("Hover over bars to see detailed information about sector allocations.")
    ], style={'margin-top': '20px'})
])

@app.callback(
    Output('aid-chart', 'figure'),
    [Input('year-range', 'value'),
     Input('view-type', 'value')]
)
def update_chart(selected_year, view_type):
    filtered_df = agg_df[agg_df['Fiscal Year'] == selected_year]
    
    if view_type == 'percentage':
        # Calculate percentages
        total_by_region = filtered_df.groupby('Region')['current_amount'].transform('sum')
        filtered_df['percentage'] = (filtered_df['current_amount'] / total_by_region) * 100
        y_value = 'percentage'
        title = f'U.S. Foreign Aid by Sector ({selected_year}) - Percentage Composition'
        y_title = 'Percentage (%)'
    else:
        y_value = 'current_amount'
        title = f'U.S. Foreign Aid by Sector ({selected_year}) - USD Amounts'
        y_title = 'Amount (USD)'
    
    fig = px.bar(
        filtered_df,
        x='Region',
        y=y_value,
        color='Sector',
        title=title,
        labels={'current_amount': 'Amount (USD)', 'percentage': 'Percentage (%)'},
        hover_data=['current_amount'],
        color_discrete_sequence=px.colors.qualitative.Pastel
    )
    
    fig.update_layout(
        barmode='stack',
        xaxis={'categoryorder': 'total descending'},
        hovermode='x unified',
        height=600,
        yaxis_title=y_title,
        xaxis_title='Region'
    )
    
    if view_type == 'percentage':
        fig.update_yaxes(range=[0, 100])
    
    return fig

if __name__ == '__main__':
    app.run(debug=True)

C:\Users\basil\AppData\Local\Temp\ipykernel_16392\4056947538.py:104: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



The data reveals significant shifts in both regional allocation and sectoral focus of U.S. foreign aid between 2014 and 2025. In 2014, the largest portion of aid went to South & Central Asia, likely supporting health initiatives and infrastructure development, followed by Europe & Eurasia where governance programs and economic reforms were probable priorities. By 2025, Europe & Eurasia emerged as the primary recipient, suggesting increased funding for security assistance and energy sector development, while South & Central Asia saw relative reductions. The aid expansion to $9 billion indicates broader sectoral investments, potentially encompassing education, climate resilience, and technology partnerships across all regions.

The visualizations employ clean, effective design choices to communicate these trends. Both charts utilize horizontal bar formats that immediately show comparative regional allocations through bar length. The consistent ordering of regions from top to bottom facilitates year-to-year comparison. While the 2014 version presents a clear hierarchy of regional priorities through simple proportional bars, the 2025 version enhances this with precise dollar values and axis markings that quantify the funding increases. The monochromatic scheme maintains focus on the data rather than decorative elements. For even greater clarity, future iterations could incorporate subtle visual cues - like segmented bars or minimal color accents - to indicate sectoral distributions within each region's total allocation.